This notebook contains the source code for the second milestone of the homework for the course  Deep Learning in Practice with Python and LUA by the team Fellow Scholars.  
Our goal is to denoise speech using Deep Learning, our choice of model architecture is a variant of wavenet, called the speech denoising wavenet.  
Our main source was this paper: https://arxiv.org/pdf/1706.07162.pdf

The members of the team are:  
Balázs Glávits  
Márk Konrád  
Andor kiss  

In [31]:
#GPU Check
!nvidia-smi

Fri Nov 20 12:57:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    36W / 250W |   5673MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
#I import the data from my Google Drive
#We use a dummy dataset, consisting of 800 noisy and 800 clean train samples and 200 noisy and 200 clean validation samples
from google.colab import drive
import numpy as np

drive.mount('/content/drive', force_remount=False)

noisy_train=np.load('/content/drive/My Drive/MLFolder/FellowScholars/noisy_train.npy')
clean_train=np.load('/content/drive/My Drive/MLFolder/FellowScholars/clean_train.npy')
noisy_val=np.load('/content/drive/My Drive/MLFolder/FellowScholars/noisy_val.npy')
clean_val=np.load('/content/drive/My Drive/MLFolder/FellowScholars/clean_val.npy')

#We didn't use our test dataset yet, because our model couldn't provide us with a satisfying validation accuracy.

#noisy_test=np.load('/content/drive/My Drive/MLFolder/FellowScholars/noisy_test.npy')
#clean_test=np.load('/content/drive/My Drive/MLFolder/FellowScholars/clean_test.npy')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(800, 119041, 1)
(200, 119041, 1)


In [33]:
#To make the data smaller, so we don't run out of ram, we slice them, and save the slices for our datagenerator.
#This is the slicing cell.
SLICE_SIZE=2048

newnoisy=[]
for file in noisy_train:
  slice_count = int(119041 / SLICE_SIZE)
  for i in range(slice_count):
    noisy_slice = file[i*SLICE_SIZE:(i+1)*SLICE_SIZE, :]
    newnoisy.append(noisy_slice)

newnoisy=np.asarray(newnoisy)


newclean=[]
for file in clean_train:
  slice_count = int(119041 / SLICE_SIZE)
  for i in range(slice_count):
    noisy_slice = file[i*SLICE_SIZE:(i+1)*SLICE_SIZE, :]
    newclean.append(noisy_slice)

newclean=np.asarray(newclean)


newnoisyval=[]
for file in noisy_val:
  slice_count = int(119041 / SLICE_SIZE)
  for i in range(slice_count):
    noisy_slice = file[i*SLICE_SIZE:(i+1)*SLICE_SIZE, :]
    newnoisyval.append(noisy_slice)

newnoisyval=np.asarray(newnoisyval)


newcleanval=[]
for file in clean_val:
  slice_count = int(119041 / SLICE_SIZE)
  for i in range(slice_count):
    noisy_slice = file[i*SLICE_SIZE:(i+1)*SLICE_SIZE, :]
    newcleanval.append(noisy_slice)

newcleanval=np.asarray(newcleanval)


In [4]:
#Here i make some directories for the saved data.
import os
os.makedirs('train/noisy')
os.mkdir('train/clean')
os.makedirs('val/noisy')
os.mkdir('val/clean')

In [34]:
#This cell is for data saving

for i, element in enumerate(newnoisy):
  np.save('train/noisy/' + str(i) + '.npy', element)


for i, element in enumerate(newclean):
  np.save('train/clean/' + str(i) + '.npy', element)


for i, element in enumerate(newnoisyval):
  np.save('val/noisy/' + str(i) + '.npy', element)


for i, element in enumerate(newcleanval):
  np.save('val/clean/' + str(i) + '.npy', element)

In [14]:
#We one-hot encode our sound samples
def onehot_256(value):
    return np.eye(256)[value]

#This is a DataGenerator class, which helps us conserve ram, by prodiving the model the data in batches.
#The code was inspired by this guide: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
from keras.utils import Sequence
class MyDatagen(Sequence):
  def __init__(self, list_IDs, batch_size=16, dim=(2048, 256), shuffle=True, validation=False):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.list_IDs = list_IDs
    self.shuffle = shuffle
    self.validation=validation
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    # Generate indexes of the batch
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

    # Find list of IDs
    list_IDs_temp = [self.list_IDs[k] for k in indexes]

    # Generate data
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  def on_epoch_end(self):
      #Updates indexes after each epoch
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    #Generates data containing batch_size samples
    if self.validation:
      X = np.empty((self.batch_size, *self.dim))
      y = np.empty((self.batch_size, *self.dim))

      # Generate data
      for i, ID in enumerate(list_IDs_temp):
        #We do the one-hot encodings here.
        X[i,] = np.squeeze(onehot_256(np.load('val/noisy/' + str(ID) + '.npy')))

        y[i,] = np.squeeze(onehot_256(np.load('val/clean/' + str(ID) + '.npy')))

      return np.squeeze((X)), np.squeeze((y))
    else:
      X = np.empty((self.batch_size, *self.dim))
      y = np.empty((self.batch_size, *self.dim))

      # Generate data
      for i, ID in enumerate(list_IDs_temp):
        X[i,] = np.squeeze(onehot_256(np.load('train/noisy/' + str(ID) + '.npy')))

        y[i,] = np.squeeze(onehot_256(np.load('train/clean/' + str(ID) + '.npy')))



      return np.squeeze((X)), np.squeeze((y))


In [35]:
#Some garbage collection if needed
import gc
gc.collect()

430

In [28]:
#This is our model, inspired by the speech denoising wavenet paper.
from keras.models import Model
from keras.layers import Conv1D, Conv2D, Flatten, Dense, \
  Input, Activation, Add, Multiply

def wavenet_residual_block(filters, kernel_size, dilation_rate):
  def f(input_):
    conv = Conv1D(filters, kernel_size,
                  dilation_rate=dilation_rate,
                  activation=None,
                  padding='same')(input_)
    tanh_out = Activation('tanh')(conv)
    sigmoid_out = Activation('sigmoid')(conv)
    merged = Multiply()([tanh_out, sigmoid_out])
    skip_out = Conv1D(1, 1, activation='relu',
                      padding='same')(merged)
    out = Add()([skip_out, input_])
    return out, skip_out
  return f

def wavenet_convolutional_layers(filters, kernel_size, depth):
  def f(input_):
    out = input_
    skip_connections = []
    for i in range(1, depth + 1):
        dilation_rate = 2**(i % 9)
        out, skip_out = wavenet_residual_block(filters, kernel_size, dilation_rate)(out)
        skip_connections.append(skip_out)
    sum = Add()(skip_connections)
    act = Activation('relu')(sum)
    return act
  return f

def wavenet(input_size=2048):
  input_ = Input(shape=(input_size, 256))
  net = wavenet_convolutional_layers(64, 2, 30)(input_)
  net = Conv1D(2048, 3, activation='relu', padding='same')(net)
  net = Conv1D(256, 3, padding='same')(net)
  #net = Flatten()(net)
  net = Dense(256, activation='softmax')(net)
  model = Model(input_, net)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  return model

In [29]:
model = wavenet(2048)

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 2048, 256)]  0                                            
__________________________________________________________________________________________________
conv1d_310 (Conv1D)             (None, 2048, 64)     32832       input_7[0][0]                    
__________________________________________________________________________________________________
activation_306 (Activation)     (None, 2048, 64)     0           conv1d_310[0][0]                 
__________________________________________________________________________________________________
activation_307 (Activation)     (None, 2048, 64)     0           conv1d_310[0][0]                 
_______________________________________________________________________________________

In [ ]:
#Here we instantiate the DataGenerators and start the training.
training_gen=MyDatagen(range(46400))
val_gen=MyDatagen(range(11600), validation=True)

model.fit(x=training_gen, validation_data=val_gen, epochs=100)

Epoch 1/100
2900/2900 [==============================] - 704s 243ms/step - loss: 4.8341 - accuracy: 0.0247 - val_loss: 4.8005 - val_accuracy: 0.0268
Epoch 2/100
2900/2900 [==============================] - 702s 242ms/step - loss: 4.7572 - accuracy: 0.0282 - val_loss: 4.7900 - val_accuracy: 0.0271
Epoch 3/100
2900/2900 [==============================] - 702s 242ms/step - loss: 4.7451 - accuracy: 0.0288 - val_loss: 4.7861 - val_accuracy: 0.0288
Epoch 4/100
2900/2900 [==============================] - 701s 242ms/step - loss: 4.7404 - accuracy: 0.0291 - val_loss: 4.7690 - val_accuracy: 0.0288
Epoch 5/100
2900/2900 [==============================] - 700s 241ms/step - loss: 4.7350 - accuracy: 0.0293 - val_loss: 4.7772 - val_accuracy: 0.0285
Epoch 6/100
2900/2900 [==============================] - 700s 241ms/step - loss: 4.7261 - accuracy: 0.0296 - val_loss: 4.7541 - val_accuracy: 0.0283
Epoch 7/100
2900/2900 [==============================] - 700s 241ms/step - loss: 4.7139 - accuracy: 0.0300